In [ ]:
!pip install pandas Pillow reportlab pystrich matplotlib


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.3 MB/s eta 0:00:00


In [ ]:
from reportlab.lib.pagesizes import A4, landscape
from reportlab.lib.units import mm
from reportlab.pdfgen import canvas
from reportlab.lib.utils import ImageReader
from pystrich.datamatrix import DataMatrixEncoder
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
import io
import re
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import pandas as pd
from google.colab import files # Importa files para Colab

# --- Parte de carga de archivo en Colab ---
print("Por favor, sube tu archivo Excel con las ubicaciones.")
uploaded = files.upload()

# Obtener el nombre del archivo subido
# Asegúrate de que solo se suba un archivo o ajusta la lógica si esperas varios
if not uploaded:
    print("No se subió ningún archivo. Por favor, sube un archivo Excel para continuar.")
    exit() # Sale del script si no hay archivos

file_name = list(uploaded.keys())[0] # Obtener el nombre del primer archivo subido
print(f"Archivo '{file_name}' subido con éxito.")

df_ubicaciones = None # Inicializar el DataFrame que usaremos

try:
    # Leer el archivo Excel
    excel_file_content = io.BytesIO(uploaded[file_name])

    # Utilizar pd.ExcelFile para obtener los nombres de las hojas y luego leer
    xls = pd.ExcelFile(excel_file_content)
    print("Nombres de las hojas en el archivo Excel:", xls.sheet_names)

    # Leer la primera hoja del archivo Excel subido
    # Asegúrate de que tu columna de ubicaciones se llame 'Ubicaciones'
    df_ubicaciones = pd.read_excel(excel_file_content, sheet_name=xls.sheet_names[0])

    print("\nDimensiones del DataFrame después de la lectura:", df_ubicaciones.shape)
    print("\nPrimeras filas del DataFrame:")
    print(df_ubicaciones.head())

    # Verificar si la columna 'Ubicaciones' existe
    if 'Ubicaciones' not in df_ubicaciones.columns:
        print("Error: La columna 'Ubicaciones' no se encontró en el archivo Excel.")
        print("Las columnas disponibles son:", df_ubicaciones.columns.tolist())
        exit() # Sale si la columna necesaria no existe

except KeyError:
    print(f"Error: No se encontró el archivo '{file_name}' entre los archivos subidos. Esto no debería ocurrir.")
    exit()
except Exception as e:
    print(f"Ocurrió un error al leer el archivo Excel: {e}")
    exit()

# --- Fin de la parte de carga de archivo en Colab ---


# Register Arial Black font (assuming it's available or using a common alternative name)
font_name = "Helvetica-Bold" # Default font for location text
try:
    # Asegúrate de que 'Arial-Black.ttf' esté en el mismo directorio o en una ruta accesible
    # En Google Colab, es posible que necesites subir este archivo de fuente también.
    pdfmetrics.registerFont(TTFont('Arial-Black', 'Arial-Black.ttf'))
    font_name = "Arial-Black" # Use Arial-Black if successfully registered
except Exception as e:
    print(f"Advertencia: No se pudo registrar la fuente 'Arial-Black'. Error: {e}. Usando fuente por defecto para texto de ubicación: {font_name}")

def create_arrow_image(direction, size_mm):
    size_inches = size_mm / 25.4
    fig, ax = plt.subplots(figsize=(size_inches, size_inches), dpi=300)
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.axis('off')

    if direction == "down":
        arrow = patches.FancyArrow(
            0.5, 0.9,  # x, y start
            0, -0.6,   # dx, dy (hacer dy menos negativo para acortar el cuerpo)
            width=0.2, head_width=0.5, head_length=0.2, fc='black', ec='black' # head_length también más corto
        )
    elif direction == "up":
        arrow = patches.FancyArrow(
            0.5, 0.1,  # x, y start
            0, 0.6,    # dx, dy (hacer dy más pequeño para acortar el cuerpo)
            width=0.2, head_width=0.5, head_length=0.2, fc='black', ec='black' # head_length también más corto
        )
    else:
        plt.close(fig) # Asegúrate de cerrar la figura incluso si no se dibuja flecha
        return None

    ax.add_patch(arrow)
    buf = io.BytesIO()
    plt.savefig(buf, format='png', transparent=True, bbox_inches='tight', pad_inches=0)
    buf.seek(0)
    plt.close(fig)
    return ImageReader(buf)


def generate_label_pdf(dataframe, output_filename="etiquetas_ubicacion.pdf"):
    c = canvas.Canvas(output_filename, pagesize=landscape(A4))
    width, height = landscape(A4)

    label_width_mm = 260
    label_height_mm = 80

    label_width_pt = label_width_mm * mm
    label_height_pt = label_height_mm * mm

    margin_x = (width - label_width_pt) / 2

    # Posiciones Y para las dos etiquetas en una página
    y_pos_top = height - (label_height_pt * 1) - 20*mm
    y_pos_bottom = height - (label_height_pt * 2) - 40*mm
    y_positions = [y_pos_top, y_pos_bottom]

    labels_per_page = 2
    label_on_page_count = 0 # Contador para saber cuántas etiquetas van en la página actual

    for index, row in dataframe.iterrows():
        ubicacion = str(row['Ubicaciones']) # Asegúrate de que el nombre de la columna sea 'Ubicaciones'

        print(f"Processing label {index + 1}: Ubicacion = {ubicacion}")

        # Si ya hemos colocado labels_per_page etiquetas, pasamos a la siguiente página
        if label_on_page_count == labels_per_page:
            c.showPage()
            label_on_page_count = 0 # Resetear el contador para la nueva página

        nivel = 0
        # Expresión regular para encontrar un dígito en la cuarta posición (índice 3)
        # ^.{3}(\d).*  -> Coincide con cualquier cosa 3 veces, captura un dígito, luego cualquier cosa
        match = re.match(r'^.{3}(\d).*', ubicacion)
        if match:
            try:
                nivel = int(match.group(1))
            except ValueError:
                print(f"Advertencia: El cuarto carácter de la ubicación '{ubicacion}' no es un dígito numérico. No se mostrará flecha.")
                nivel = 0
        else:
            print(f"Advertencia: La ubicación '{ubicacion}' no tiene un cuarto carácter numérico para determinar el nivel. No se mostrará flecha.")
            nivel = 0

        print(f"Nivel determinado: {nivel}")

        current_x = margin_x
        # Usar el contador de etiquetas en la página para elegir la posición Y
        current_y = y_positions[label_on_page_count]

        # 1. Código Data Matrix
        encoder = DataMatrixEncoder(ubicacion)
        datamatrix_img_data = encoder.get_imagedata()
        datamatrix_image = ImageReader(io.BytesIO(datamatrix_img_data))

        dm_size_mm = 60
        dm_size_pt = dm_size_mm * mm
        dm_x = current_x
        dm_y = current_y + (label_height_pt / 2) - (dm_size_pt / 2)
        c.drawImage(datamatrix_image, dm_x, dm_y, width=dm_size_pt, height=dm_size_pt)

        # 2. Texto de Ubicación
        font_size = 80
        c.setFont(font_name, font_size)
        text_ubicacion_x = current_x + (label_width_pt / 2)
        text_height = font_size * 0.8
        text_ubicacion_y = current_y + (label_height_pt / 2) - (text_height / 2)
        c.drawCentredString(text_ubicacion_x, text_ubicacion_y, ubicacion)

        # 3. Flecha (condicionalmente) usando Matplotlib
        arrow_image = None
        arrow_size_mm = 50 # Tamaño de la flecha en mm
        arrow_size_pt = arrow_size_mm * mm

        if nivel == 1:
            arrow_image = create_arrow_image("down", arrow_size_mm)
        elif nivel == 2:
            arrow_image = create_arrow_image("up", arrow_size_mm)

        if arrow_image:
            arrow_x = current_x + label_width_pt - arrow_size_pt - 10 * mm
            arrow_y = current_y + (label_height_pt / 2) - (arrow_size_pt / 2)
            c.drawImage(arrow_image, arrow_x, arrow_y, width=arrow_size_pt, height=arrow_size_pt, mask='auto')

        label_on_page_count += 1 # Incrementar el contador de etiquetas en la página actual

    c.save()
    print(f"PDF generado: {output_filename}")

# --- Llamada a la función principal ---
# Solo llama a generate_label_pdf si el DataFrame se cargó correctamente
if df_ubicaciones is not None:
    generate_label_pdf(df_ubicaciones, output_filename="etiquetas_ubicacion.pdf")
else:
    print("No se pudo generar el PDF porque no se cargaron los datos de ubicaciones.")

Por favor, sube tu archivo Excel con las ubicaciones.


Saving ubicaciones.xlsx to ubicaciones.xlsx
Archivo 'ubicaciones.xlsx' subido con éxito.
Nombres de las hojas en el archivo Excel: ['Sheet1']

Dimensiones del DataFrame después de la lectura: (1160, 1)

Primeras filas del DataFrame:
  Ubicaciones
0       61A1A
1       61A2A
2       61A3A
3       61B1A
4       61B2A
Advertencia: No se pudo registrar la fuente 'Arial-Black'. Error: Can't open file "Arial-Black.ttf". Usando fuente por defecto para texto de ubicación: Helvetica-Bold
Processing label 1: Ubicacion = 61A1A
Nivel determinado: 1
Processing label 2: Ubicacion = 61A2A
Nivel determinado: 2
Processing label 3: Ubicacion = 61A3A
Nivel determinado: 3
Processing label 4: Ubicacion = 61B1A
Nivel determinado: 1
Processing label 5: Ubicacion = 61B2A
Nivel determinado: 2
Processing label 6: Ubicacion = 61B3A
Nivel determinado: 3
Processing label 7: Ubicacion = 61C2A
Nivel determinado: 2
Processing label 8: Ubicacion = 61C3A
Nivel determinado: 3
Processing label 9: Ubicacion = 61D1A
Nivel 

In [ ]:
from google.colab import files

files.download('etiquetas_ubicacion.pdf')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>